# Advanced Problems in Computer Vision

So far typical computer vision problems revolved around object detection, classification and segmentation. But there are some other problems that require advanced techniques of computer vision to solve.

## Object Measurement

In many cases, we not only need to identify the object, we need to measure its dimensions or some other metric. For example we might need to create an application to measure the feet size based on image. In these type of use cases we cant just calculate the number of pixels. We need to have a proper methodology to accurately measure the values in varies scales.

One of the most simplest way to solve this problem is that having a reference object in the frame. If we can premeasure the size of that reference object, then we can use it to measure the unknown object dimensions once we detect it properly. 

For the detection work, first we can do a segmentation task using a Mask-RCNN like model. That way we can clearly separate the reference object and interested object from the background. Then we have to do a rotation correction to make sure the object we are trying to measure is in correct orientation. To do that we can use PCA (Principal Component Analysis) and get the eigen vectors for segmentation masks. Then the largest eigen vector magnitude is the size of object we are trying to measure in its longest direction. (This is because eigen vectors represent the field in orthogonal axes with their highest variance representations). 

Also since PCA axes are orthogonal and in a different space which have its axes aligned with object of interest's direction of greatest variance, we can use it to measure how much we need to rotate the original image to get the correct rotation.

The complexity of this task comes in combining above mentioned techniques in to a single working solution based on the usecase.


## Counting

Another interesting complex problem in computer vision is object counting. There are many applications that could be enhanced by solving this problem properly.

There are multiple ways to tackle this problem. Based on the use case we need to find a suitable method.

1. Training a object detection model with boubdaries and count the number of bounding boxes. But if theres items with overlapping areas it might cause problems during counting.

2. Treating the problem as a segmentation use case. We can segment each object in to separate masks/clusters. But problem is that, most of the time segmentation techniques are not scale invarient. Therefore object we try to count may not get segmented if it is too small or large.

3. Treat this as a regression problem and try to directly infer the number of objects from the original image. Though this may seem more simple, it suffers from the scale related issues just as in segmentation technnique and also it is hard to find a proper dataset as well. 

Other than above 3, another interesting solution for this problem is a technique called density approximation. 

Idea here is to train a model to estimate the density of objects inside patches of the image. In order to train a model to get density, we need to have properly labeled dataset which include the density values. 

So to prepare such dataset, we should first take the original image and break it into non overlapping  patches. Then for each of those patches we need to calculate the density. In cases of only a part of considering object is present we can consider a unique point in the object and count if it is inside of a patch. Read more about this technique [here](https://papers.nips.cc/paper/2010/file/fe73f687e5bc5280214e0486b273a5f9-Paper.pdf).

## Pose Estimation

Another interesting usecase of computer vision is Pose Estimation. Here we try to identify key parts of objects and how they are aligned with respect to each other(pose). It is useful to identify the actions/state of an object (like for human sitting, running etc.)

To do this task in 2018 a novel technique called [PersonLab](https://arxiv.org/pdf/1803.08225.pdf) was introduced. Now models that implement this technique is now go by the name PoseNet and its overall idea is as below.

<center><image src="./imgs/24.png" width="400px"/></center>

1. Use an object detection model to identify key points heatmap of the interested object (For example for human subject these keypoints would be knees, elbows, shoulders like joints).

2. For each identified point, find the most likely location of nearby point.

3. Use a voting mechanism to detect the pose based on points chosen from the above 2 steps.

In practise, step 1 and 2 are being done using a single step. The reason for having a nearby point estimation step is because, there may be cases where model will not be able to identify the required points and in such cases voting mechanism can help to estimate the point location.

An example of using posenet model can be found [here](https://www.tensorflow.org/lite/examples/pose_estimation/overview).

For the cases of identifying multiple poses we can use the same technique outlined with few additional steps.

1. Use an image segmentation model to identify all the pixels corresponding to the objects/items of interest.

2. Using the previously mentioned like technique, identify the most likely location of a point of interest.

3. Using the pixels from the segmentation map and most likely connected points of interests of step 2, assign object ids to individual objects/items. 

## Image Search

Image search is a very interesting use case of computer vision where we try to identify similar images to the pivot image. Core concept of achieving this task is image embeddings. We can then use those embeddings to identify similar images.

But to do this we need to create a search index of embeddings. We can use SQL or some other service like Google BigQuery to store and search the dataset. But in cases where the dataset is extremely large, getting similar embeddings may take considerable amount of time. This is unacceptable for real-time serving purposes. Therefore we can use techniques like Scalable Nearest Neighbours (ScaNN) method, which provides an efficient way of finding similar vectors.